# Single Shot Multibox Detection(SSD)
배웠던 것을 활용하여 object detection model SSD를 만들어보자.  
SSD는 널리 이용되고 있으며 다른 object detection model에도 적용 가능하다.  
읽어볼 것 : :cite:`Liu.Anguelov.Erhan.ea.2016`

## Model
SSD 모델의 주 요소는 base network와 연속적으로 연결된 다규모의 feature block이다.  
Base network 블록은 원본 이미지의 feature를 추출하는 데 사용되고 deep convolutional neural network의 형태를 갖는다(그러나 최근 ResNet으로 대체되는 추세다?).

## Category Prediction Layer
Object category 개수가 $q$면 anchor box category 개수는 $q+1$이 된다. Background만을 포함하는 anchor box의 number는 0이 되는 것이다. Classifying에 FCN을 사용하게 되면 computational burden이 너무 커서 SSD는 model complexity를 감소시킬 필요가 있다.

Category prediction layer에서는 input의 h, w를 유지하는 convolutional layer를 사용한다. 즉 output과 input의 좌표는 feature map의 그것과 일대일 대응한다.
$a$와 $q$ parameter를 명시한 후에 padding 1, 3 by 3 convolutional layer를 사용한다. Input과 output의 해상도는 변하지 않는다.

In [2]:
%matplotlib inline
import sys
sys.path.insert(0, '..')
import d2l
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import json
import time
from tqdm import tqdm
from PIL import Image
def cls_predictor(input_channels, num_anchors, num_classes):
    return nn.Conv2d(in_channels=input_channels, out_channels=num_anchors * (num_classes + 1), kernel_size = 3, padding = 1)

## Bounding Box Prediction Layer
Bounding box prediction layer 설계는 category prediction layer의 그것과 유사하다. 차이점은 anchor box 당 $q+1$이 아닌 4개의 offset이 필요하다는 점이다.

In [3]:
def bbox_predictor(input_channels, num_anchors):
    return nn.Conv2d(in_channels=input_channels, out_channels=numanchors * 4, kernel_size = 3, padding = 1)

## Concatenating Predictions for Multiple Scales
